# Test Neural Network Predictive Abilities on Unseen Atoms in CrFeCoNiCu
<b> nanoHUB tools by: </b>  <i>Mackinzie S. Farnell, Zachary D. McClure</i> and <i>Alejandro Strachan</i>, Materials Engineering, Purdue University <br>

We use models trained on FeCoNiCu, CrCoNiCu, CrFeNiCu, CrFeCoCu, and CrFeCoNi to predict relaxed vacancy formation energy, cohesive energy, pressure, and volume for CrFeCoNiCu. The purpose of these tests is to see how the model predicts unseen atoms (i.e. atoms not in the training set) and if the specific atom missing from the training set affects the results. 

Overview
1. Load Bispectrum Coefficients and Output Properties
2. Add Pymatgen Descriptors
3. Normalize Properties
4. Predict Properties and Evaluate Model
5. Discuss Results

In [ ]:
#import libraries we will need
import tensorflow as tf
import keras as ke
from keras.models import load_model

import json as js
import numpy as np
import pymatgen as pymat
import csv
import re

import plotly.offline as p
import plotly.graph_objs as go

## 1. Load Bispectrum Coefficients and Output Properties
The unrelaxed bispectrum coefficients and output properties for CrFeCoNiCu are stored in a JSON file. The code below extracts this information from the json file and stores each property in a numpy array. These arrays are stored in a dictionary for easy access. We are testing on models trained with only the bispectrum coefficients because additional descriptors lead the model to predict poorly on the unseen atom.

In [ ]:
properties = ["Relaxed_VFE", "Cohesive_Energy", "Pressure", "Volume"] 
filename = '../data/20_20_20_20_20.json'
input_prop_key = 'Unrelaxed_Bispectrum_Coefficients'

properties_dictionaries = {}

for output_prop_key in properties:
    # open file and load data into data variable
    with open(filename, 'r') as f:
        data = js.load(f)

    # get relevant information from data variable
    elements = data['element']
    output_properties = data[output_prop_key]
    input_properties = data[input_prop_key]

    # store input and output properties for specific element being searched for
    elements_array = np.array([]) 
    output_properties_array = np.array([])
    input_properties_array = np.array([])

    # create counters to track number of each element
    num_Cr = 0
    num_Fe = 0
    num_Co = 0
    num_Ni = 0
    num_Cu = 0
    
    # iterate through elements and get input and output properties for desired element
    for i, val in enumerate(elements):
        output_properties_array = np.append(output_properties_array, output_properties[i])
        input_properties_array = np.append(input_properties_array, np.asarray(input_properties[i])) 
        if (val == 'Cr'):
            elements_array = np.append(elements_array, 24)
            num_Cr = num_Cr + 1
        elif (val == 'Fe'):
            elements_array = np.append(elements_array, 26)
            num_Fe = num_Fe + 1
        elif (val == 'Co'):
            elements_array = np.append(elements_array, 27)
            num_Co = num_Co + 1
        elif (val == 'Ni'):
            elements_array = np.append(elements_array, 28)
            num_Ni = num_Ni + 1
        elif (val == 'Cu'):
            elements_array = np.append(elements_array, 29)
            num_Cu = num_Cu + 1
 
    # reshape input_properties_element
    num_rows = int (input_properties_array.shape[0]/55)
    input_properties_array = np.reshape(input_properties_array, (num_rows, 55))
    
    # element number is included as input to model
    elements_array = elements_array[np.newaxis].T
    input_properties_array = np.append(input_properties_array, elements_array, 1)

    input_properties_array = input_properties_array.astype(np.float)

    if (output_prop_key == 'Relaxed_VFE'):
        min_val = 0.9
        max_val = 2.3
        display = 'Relaxed VFE (eV/atom)'
        units = '(eV/atom)'
        file_id = 'relaxed_vfe'
        prop = 'Relaxed_VFE'
    elif (output_prop_key == 'Cohesive_Energy'):
        min_val = -5
        max_val = -3
        display = 'Cohesive Energy (eV/atom)'
        units = '(eV/atom)'
        file_id = 'cohesive_energy'
        prop = 'Cohesive_Energy'
    elif (output_prop_key == 'Pressure'):
        min_val = -7
        max_val = 7
        display = 'Pressure (GPa)'
        units = '(GPa)'
        file_id = 'pressure_bs_centr'
        prop = 'Pressure'
    elif (output_prop_key == 'Volume'):
        min_val = 10.9
        max_val = 11.8
        display = 'Volume (\u212B\u00b3)'
        units = '(\u212B\u00b3)'
        file_id = 'volume_bs_centr'
        prop = 'Volume'

    # create properties_dictionary to return to the user
    properties_dictionaries[output_prop_key] = {
        'inputs': input_properties_array,
        'outputs': output_properties_array,
        'length': output_properties_array.shape[0],
        'elements': elements_array,
        'min': min_val,
        'max': max_val,
        'display': display,
        'units': units,
        'file_id': file_id
    }

## 2. Add Pymatgen Descriptors

Properties are queried from Pymatgen to use as model inputs along with the bispectrum coefficients. We add central atom descriptors because the output property values vary based on which atom we are predicting on and the central atom descriptors give the neural network a way to distinguish between different atoms. The queried properties are:
    - atomic_radius_calculated
    - atomic_radius 
    - atomic_mass
    - poissons_ratio
    - electrical_resistivity
    - thermal_conductivity
    - brinell_hardness

In [ ]:
# declare function to query property from pymatgen for a given element
def get_property(element, property):
    element_object = pymat.Element(element)
    element_prop = getattr(element_object, property)
    return element_prop

# list of properties to add central atom descriptors for
properties = ['atomic_radius_calculated', 'atomic_radius', 'atomic_mass', 
              'poissons_ratio', 'electrical_resistivity', 'thermal_conductivity', 
              'brinell_hardness']

# iterate through all output properties
for key in properties_dictionaries:
      
    # iterate through all properties to add
    for add_property in properties:
        atom_properties = []
        elements = properties_dictionaries[key]['elements']
        # determine which element to get property for
        for i in elements:
            if (i == 24):
                ele = 'Cr'
            elif (i == 26):
                ele = 'Fe'
            elif (i == 27):
                ele = 'Co'
            elif (i == 28):
                ele = 'Ni'
            elif (i == 29):
                ele = 'Cu'
                
            prop = get_property(ele, add_property)
            atom_properties.append(float (prop))

        # add property to array of inputs
        atom_properties = np.asarray(atom_properties) 
        atom_properties = atom_properties[np.newaxis].T
        properties_dictionaries[key]['inputs'] = np.append(properties_dictionaries[key]['inputs'], 
                                                           atom_properties, 1)

## 3. Normalize Properties
We normalize the inputs and outputs to the model using mean and standard deviation. Each data point (x) is normalized using the mean ($ \mu $) and standard deviation ($ \sigma $) of the set of values. We normalize the data using the mean and standard deviation of the training data for each of the 4 atom systems (FeCoNiCu, CrCoNiCu, CrFeNiCu, CrFeCoCu, CrFeCoNi) so that the models predict on points from the distributions they were trained on. This means we have 5 sets of normalized CrFeCoNiCu values that we will use to test the 5 models trained on FeCoNiCu, CrCoNiCu, CrFeNiCu, CrFeCoCu, and CrFeCoNi.

$$ x_{new} = \frac{x - µ}{σ}\ $$

In [ ]:
# define helper function to normalize data points
def normalize(test_train_properties, key, means, stdevs):
    dims = test_train_properties[key].shape

    for j in range(0, dims[0]):
        test_train_properties[key][j] = (test_train_properties[key][j] - means)/stdevs
  
    test_train_properties[key] = np.nan_to_num(test_train_properties[key])
    
    return test_train_properties

In [ ]:
# list all for atom systems
atom_systems = ['FeCoNiCu', 'CrCoNiCu', 'CrFeNiCu', 'CrFeCoCu', 'CrFeCoNi']

for key in properties_dictionaries:
    # iterate through all four atom systems
    for sys in atom_systems:
        stats_file_name = "my_models/63/{}_stats_{}.json".format(properties_dictionaries[key]['file_id'], sys)
        # load the stats dictionary
        f = open(stats_file_name,"r")
        stats_data = js.load(f)
        f.close()
  
        for num in stats_data:
            stats_data[num] = np.asarray(stats_data[num])

        properties_dictionaries[key]['stats-{}'.format(sys)] = stats_data    
        dims = properties_dictionaries[key]["inputs"].shape   
        properties_dictionaries[key]["inputs-{}".format(sys)] = \
                                np.zeros(np.shape(properties_dictionaries[key]["inputs"]))

        # normalize
        for j in range(0, dims[0]):
            properties_dictionaries[key]["inputs-{}".format(sys)][j] = ((properties_dictionaries[key]["inputs"][j] - 
                                                                  stats_data["means_ins"])/stats_data["stdevs_ins"])

        properties_dictionaries[key]["inputs-{}".format(sys)] = \
                                np.nan_to_num(properties_dictionaries[key]["inputs-{}".format(sys)])


## 4. Predict Properties and Evaluate Model
We use the model trained on FeCoNiCu, CrCoNiCu, CrFeNiCu, CrFeCoCu, and CrFeCoNi to predict relaxed vacancy formation energy of CrFeCoNiCu.

In [ ]:
for key in properties_dictionaries:
    for sys in atom_systems:
        model = load_model("my_models/63/{}_model_{}.h5".format(properties_dictionaries[key]['file_id'], sys))
        properties_dictionaries[key]['predict-{}'.format(sys)] = \
                                model.predict(properties_dictionaries[key]["inputs-{}".format(sys)])

Models are evaluated by plotting the predicted versus actual data and calculating the (mean absolute error) MAE and (mean squared error) MSE of the test predictions (equations are shown below). We divide the MAE and MSE by the range for each property so that we can compare the error for different output properties.

$$ MSE = \frac{\frac{1}{n}\sum\limits _{i=1} ^{n}(Y_{i}-\hat{Y}_{i})^2}{max-min} $$


$$ MAE = \frac{\frac{1}{n}\sum\limits _{i=1} ^{n}|Y_{i}-\hat{Y}_{i}|}{max-min} $$

In [ ]:
for key in properties_dictionaries:
    for sys in atom_systems:
    
        #write data
        actual_data_points = properties_dictionaries[key]["outputs"]
        predicted_data_points = (properties_dictionaries[key]['predict-{}'.format(sys)] 
                            * properties_dictionaries[key]['stats-{}'.format(sys)]["stdevs_outs"] 
                            + properties_dictionaries[key]['stats-{}'.format(sys)]["means_outs"])

        md_data_Cr = []
        nn_data_Cr = []
        md_data_Fe = []
        nn_data_Fe = []
        md_data_Co = []
        nn_data_Co = []
        md_data_Ni = []
        nn_data_Ni = []
        md_data_Cu = []
        nn_data_Cu = []

        # separate out data for each element
        for i, val in enumerate(properties_dictionaries[key]['elements']):
            if (val == 24):
                md_data_Cr.append(actual_data_points[i])
                nn_data_Cr.append(predicted_data_points[i])
            elif (val == 26):
                md_data_Fe.append(actual_data_points[i])
                nn_data_Fe.append(predicted_data_points[i])
            elif (val == 27):
                md_data_Co.append(actual_data_points[i])
                nn_data_Co.append(predicted_data_points[i])
            elif (val == 28):
                md_data_Ni.append(actual_data_points[i])
                nn_data_Ni.append(predicted_data_points[i])
            elif (val == 29):
                md_data_Cu.append(actual_data_points[i])
                nn_data_Cu.append(predicted_data_points[i])

        md_data_Cr = np.array(md_data_Cr)
        nn_data_Cr = np.array(nn_data_Cr)
        nn_data_Cr = nn_data_Cr.flatten()


        md_data_Fe = np.array(md_data_Fe)
        nn_data_Fe = np.array(nn_data_Fe)
        nn_data_Fe = nn_data_Fe.flatten()
        
        md_data_Co = np.array(md_data_Co)
        nn_data_Co = np.array(nn_data_Co)
        nn_data_Co = nn_data_Co.flatten()

        md_data_Ni = np.array(md_data_Ni)
        nn_data_Ni = np.array(nn_data_Ni)
        nn_data_Ni = nn_data_Ni.flatten()
        
        md_data_Cu = np.array(md_data_Cu)
        nn_data_Cu = np.array(nn_data_Cu)
        nn_data_Cu = nn_data_Cu.flatten()
    
        dot_size = 6

        # make plots
        fig = go.Figure()
        x_lin = [-24, 24]
        fig.add_trace(go.Scatter(x=x_lin, 
                         y=x_lin,
                         mode='lines',
                         line=dict(color="black"),
                         showlegend=False)
             )
        fig.add_trace(go.Scatter(x=md_data_Cr,
                         y=nn_data_Cr,
                         legendgroup="2",
                         name="Cr",
                         mode='markers',
                         marker=dict(
                            color="red",
                            size=dot_size
                            )
                        )
             )
        fig.add_trace(go.Scatter(x=md_data_Fe,
                         y=nn_data_Fe,
                         legendgroup="3",
                         name="Fe",
                         mode='markers',
                         marker=dict(
                            color="orange",
                            size=dot_size
                            )
                        )
             )
        fig.add_trace(go.Scatter(x=md_data_Co,
                         y=nn_data_Co,
                         legendgroup="1",
                         name="Co",
                         mode='markers',
                         marker=dict(
                            color="blue",
                            size = dot_size
                            )
                        )
             )
        fig.add_trace(go.Scatter(x=md_data_Ni,
                         y=nn_data_Ni,
                         legendgroup="4",
                         name="Ni",
                         mode='markers',
                         marker=dict(
                            color="green",
                            size=dot_size
                            )
                        )
             )
        fig.add_trace(go.Scatter(x=md_data_Cu,
                         y=nn_data_Cu,
                         legendgroup="2.5",
                         name="Cu",
                         mode='markers',
                         marker=dict(
                            color="darkgrey",
                            size=dot_size
                            )
                        )
             )

        fig.update_xaxes(range=[properties_dictionaries[key]['min'], properties_dictionaries[key]['max']])
        fig.update_yaxes(range=[properties_dictionaries[key]['min'], properties_dictionaries[key]['max']])

        fig.update_layout(
            showlegend=True,
            title="{} - {}". format(key, sys),
            xaxis_title="Molecular Mechanics {}".format(properties_dictionaries[key]['units']),
            yaxis_title="Neural Network {}".format(properties_dictionaries[key]['units']),
            font=dict(
                family="Times New Roman, monospace",
                size=24,
                color= "black"
            )
        )
        fig.show()

        # calculate error
        test_mse = np.mean((predicted_data_points-actual_data_points)**2)
        test_mae = np.mean(np.abs(predicted_data_points-actual_data_points))
        test_error = (predicted_data_points-actual_data_points)

        # find a normalized mse and mae
        max = np.amax(actual_data_points)
        min = np.amin(actual_data_points)
        test_range = np.abs(max - min)
        mse_norm = test_mse/test_range
        mae_norm = test_mae/test_range

        print(f'Test_MAE/range: {mae_norm:.5f}')
        print(f'Test_MSE/range: {mse_norm:.5f}')  

## 5. Discuss Results
Based on these results, there a few observations we can make about the neural network's predictive abilities on unseen atoms. 
1. The network predicts poorly when trained on FeCoNiCu (without Cr). The predictions form a globular shape, rather than falling along the x=y line.
2. The network predictions are generally good when trained on CrCoNiCu (without Fe), CrFeNiCu (without Co), and CrFeCoCu (without Ni).
3. The network predicts poorly on the Cu atom when trained on CrFeCoNi (without Cu). 

To determine the reason for these poor predictions, we compare the bispectrum coefficients of each 4-atom system to CrFeCoNiCu. We observe that the bispectrum coefficients for FeCoNiCu are not similar to the bispectrum coefficients for CrFeCoNiCu. The plots below compare the first bispectrum coefficient of each 4-atom system with CrFeCoNiCu. For FeCoNiCu and CrFeCoNiCu, the range of values and peak locations are different. You can view plots for the other bispectrum coefficients and the code to generate these plots in [this notebook](BS-histograms.ipynb).

<p float="left">
  <img src=compare-bs-coeffs.jpg width="700px" height='300px' align="center" /> 
</p>

The poor predictions on Cu when trained on CrFeCoNi result from the central atom descriptors affecting the predictions. The neural network has better predictive abilities on Cu when trained only with bispectrum coefficients. Please see [this notebook](test_CrFeCoNiCu_predict_bs.ipynb) to view the results of training and testing the neural network with only bispectrum coefficients.

In [ ]:
print("done:)")